<a href="https://colab.research.google.com/github/masatokg/AI_teach2020/blob/master/%E3%80%90template%E3%80%912020_WebCam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## GoogleColabでOpenCV＆webCamera
### GoogleColab上でWebCameraとOpenCVを連携させたい時にどうぞ
### 書き換えた時はランタイムを初期化しないとブラウザに前のキャッシュが残っている可能性あります

### Cell01

In [ ]:
from IPython.display import display, Javascript
# GoogleのPythonからのJavaScript呼び出しライブラリ
from google.colab.output import eval_js 

# メイン処理関数
def use_cam(quality=0.8):
  js = Javascript('''
    async function useCam(quality) {
      const div = document.createElement('div');
      document.body.appendChild(div);
      //video element
      const video = document.createElement('video');
      video.style.display = 'None';

      let media_array = [];
      let i = 0;

      // 【masatokg】mediaDeciveの情報をページに表示（デバッグ用）
      /*
      navigator.mediaDevices.enumerateDevices()
        .then(devices => {
          mediaDeviceInfo = devices;
          var h = document.createElement("H1");
          document.body.appendChild(h);
          for( device of devices){
            media_array[i] = device['deviceId'];
            var t = document.createTextNode(" device_no:"+i + JSON.stringify(device) + "media_array "+i +": "+ media_array[i]);
            h.appendChild(t);
            i++;
          }
        });
      */

      const stream = await navigator.mediaDevices.getUserMedia({video:true} );
      // deviceIdを決め打ちする場合、以下のようにする
      // const stream = await navigator.mediaDevices.getUserMedia({video: { deviceId: "cf908ea160fd9ca5735e336884408515744c0dc7311fd02b33056ac9ae581a31" },} );

      // オリジナル画面を映すビデオ要素を画面に追加
      div.appendChild(video);
      video.srcObject = stream;
      await video.play();

      //canvas for display. frame rate is depending on display size and jpeg quality.
      // display_size = 640 // original
      display_size = 320 // 小さめにしました
      const src_canvas = document.createElement('canvas');
      src_canvas.width  = display_size;
      src_canvas.height = display_size * video.videoHeight / video.videoWidth;
      const src_canvasCtx = src_canvas.getContext('2d');
      src_canvasCtx.translate(src_canvas.width, 0);
      src_canvasCtx.scale(-1, 1);
      div.appendChild(src_canvas);

      // cvの画面を映すキャンバス要素を画面に追加
      const dst_canvas = document.createElement('canvas');
      dst_canvas.width  = src_canvas.width;
      dst_canvas.height = src_canvas.height;
      const dst_canvasCtx = dst_canvas.getContext('2d');
      div.appendChild(dst_canvas);

      //exit button 描画中止ボタン
      const btn_div = document.createElement('div');
      document.body.appendChild(btn_div);
      const exit_btn = document.createElement('button');
      exit_btn.textContent = 'Camera Stop';
      var exit_flg = true
      exit_btn.onclick = function() {exit_flg = false};
      btn_div.appendChild(exit_btn);

      // Resize the output to fit the video element.
      google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

      var send_num = 0
      // loop
      _canvasUpdate();
      async function _canvasUpdate() {
            // 【masatokg】元映像のCanvasにVideoオブジェクトのStreamを描画
            src_canvasCtx.drawImage(video, 0, 0, video.videoWidth, video.videoHeight, 0, 0, src_canvas.width, src_canvas.height);     
            if (send_num<1){
                send_num += 1
                // 【masatokg】Canvasの描画イメージを取得
                const img = src_canvas.toDataURL('image/jpeg', quality);
                /* 【masatokg】GoogleColabに登録した自作のPythonCallback 'notebook.run' をJavaScriptから呼び出す(引数は画像のBase64)
                 https://github.com/googlecolab/colabtools/issues/6
                 https://github.com/googlecolab/colabtools/blob/master/google/colab/output/_js.py#L45
                 */
                const result = google.colab.kernel.invokeFunction('notebook.run', [img], {});
                // 【masatokg】Callback の戻値（画像加工済のBase64）を匿名関数に引き渡す
                result.then(function(value) {
                    // 【masatokg】JSONオブジェクトを解析して画像Base64をイメージソースに指定してcv画像のCanvasに描画
                    parse = JSON.parse(JSON.stringify(value))["data"]
                    parse = JSON.parse(JSON.stringify(parse))["application/json"]
                    parse = JSON.parse(JSON.stringify(parse))["img_str"]
                    var image = new Image()
                    image.src = parse;
                    image.onload = function(){dst_canvasCtx.drawImage(image, 0, 0)}
                    send_num -= 1
                })
            }
            if (exit_flg){
                requestAnimationFrame(_canvasUpdate);   
            }else{
                stream.getVideoTracks()[0].stop();
            }
      };
    }
    ''')
  # JavaScriptを画面HTMLに読込表示
  display(js) 
  # PythonからJavaScript関数(useCam)をeval_jsで実行
  data = eval_js('useCam({})'.format(quality))
# use_cam定義完了

### Cell02
Python側のメイン処理（use_cam）から呼び出す todo処理
ここをやりたいことに応じて変化させる
コールバック関数（毎回呼び出される関数）をセットする前に1回だけやりたい処理をこのタイミングでやっておく部分

In [ ]:
import cv2


### Cell03 
Python側のメイン処理（use_cam）から呼び出す todo処理
ここをやりたいことに応じて変化させる
コールバック関数で毎回呼び出されて処理する部分

In [ ]:
def doDef(decimg):


  #【masatokg】好きなPython処理を記述する。
  #############your process###############

  out_img = decimg

  #############your process###############
  return out_img  # 画像をかえす
# 関数定義終了


### Cell04 Python側のメイン処理（use_cam）定義部分

In [ ]:
import IPython
from google.colab import output
from google.colab.patches import cv2_imshow # incompatible with Jupyter notebook
import cv2
import numpy as np
from PIL import Image, ImageDraw
from io import BytesIO
import base64

# JavaScript から呼び出すためのコールバック関数を定義（引数は画像のBase64）
def run(img_str):
    #decode to image 【masatokg】base64を解析してイメージに変換
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8); 
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    # ここから下を関数（doDef）化 
    # 元画像を引き渡してPython処理後、加工画像を受け取る
    out_img = doDef(decimg)

    #encode to string
    _, encimg = cv2.imencode(".jpg", out_img, [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tostring()
    img_str = "data:image/jpeg;base64," + base64.b64encode(img_str).decode('utf-8')
    # 加工後の画像をbase64でコールバックの戻り値とする
    return IPython.display.JSON({'img_str': img_str})
# 関数 run の定義完了

# 自作した関数 run をcall back名 'notebook.run' でPythonに登録（画面のJavaScriptから呼び出すつもり）
output.register_callback('notebook.run', run)

### Cell05 Pythonの実行関数呼び出しでJavaScriptでHTML組み立て＆表示を実行

In [ ]:
use_cam()

<IPython.core.display.Javascript object>